## Deliverable 2. Create a Customer Travel Destinations Map.

In [7]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [8]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeartherPy_Database.csv")
city_data_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Kem,RU,64.9555,34.5793,66.83,82,96,7.34,overcast clouds
1,1,New Norfolk,AU,-42.7826,147.0587,49.71,83,100,1.01,overcast clouds
2,2,Bredasdorp,ZA,-34.5322,20.0403,53.85,64,100,8.50,overcast clouds
3,3,Eirunepe,BR,-6.6603,-69.8736,84.65,70,67,1.05,broken clouds
4,4,Punta Arenas,CL,-53.1500,-70.9167,41.02,75,0,20.71,clear sky
5,5,Kapaa,US,22.0752,-159.3190,80.58,83,13,3.00,few clouds
6,6,Upernavik,GL,72.7868,-56.1549,41.99,95,100,24.25,light rain
7,7,Coquimbo,CL,-29.9533,-71.3436,55.13,82,100,2.30,overcast clouds
8,8,Hermanus,ZA,-34.4187,19.2345,53.80,71,99,10.49,overcast clouds
9,9,Ushuaia,AR,-54.8000,-68.3000,35.26,93,100,29.93,sleet


In [9]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 

min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 95


In [10]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_city_data_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,3,Eirunepe,BR,-6.6603,-69.8736,84.65,70,67,1.05,broken clouds
5,5,Kapaa,US,22.0752,-159.3190,80.58,83,13,3.00,few clouds
13,13,Atuona,PF,-9.8000,-139.0333,77.07,75,0,14.45,clear sky
18,18,Lardos,GR,36.0942,28.0157,89.42,51,0,6.26,clear sky
19,19,Port Blair,IN,11.6667,92.7500,80.80,84,100,7.05,light rain


In [11]:
# 4a. Determine if there are any empty rows.
preferred_city_data_df.isnull().sum()

# OR: 
# preferred_city_data_df.count()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [12]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.

# There were no empty rows, but if there were we would we would use the below:

clean_df = preferred_city_data_df.dropna()

In [13]:
clean_df.isnull().sum()

# OR: 
# new_df.count()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [14]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Eirunepe,BR,84.65,broken clouds,-6.6603,-69.8736,
5,Kapaa,US,80.58,few clouds,22.0752,-159.3190,
13,Atuona,PF,77.07,clear sky,-9.8000,-139.0333,
18,Lardos,GR,89.42,clear sky,36.0942,28.0157,
19,Port Blair,IN,80.80,light rain,11.6667,92.7500,
21,Lorengau,PG,81.43,broken clouds,-2.0226,147.2712,
22,Bathsheba,BB,86.61,scattered clouds,13.2167,-59.5167,
25,Muros,ES,76.68,clear sky,42.7762,-9.0603,
26,Albany,US,92.62,few clouds,42.6001,-73.9662,
29,Lloret De Mar,ES,89.11,scattered clouds,41.6999,2.8457,


In [15]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [16]:
# 6b. Iterate through the hotel DataFrame.

for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search. 
   
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [17]:
# 7. Drop the rows where there is no Hotel Name.

clean_hotel_df = hotel_df[hotel_df["Hotel Name"] != ""]
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Eirunepe,BR,84.65,broken clouds,-6.6603,-69.8736,Hotel novo Acre & Comercial Aguiar
5,Kapaa,US,80.58,few clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
13,Atuona,PF,77.07,clear sky,-9.8000,-139.0333,Villa Enata
18,Lardos,GR,89.42,clear sky,36.0942,28.0157,Lindos Princess Beach Hotel
19,Port Blair,IN,80.80,light rain,11.6667,92.7500,"Welcomhotel By ITC Hotels, Bay Island, Port Blair"
21,Lorengau,PG,81.43,broken clouds,-2.0226,147.2712,Lorengau Harbourside Hotel
22,Bathsheba,BB,86.61,scattered clouds,13.2167,-59.5167,Atlantis Hotel
25,Muros,ES,76.68,clear sky,42.7762,-9.0603,Apartment Av. Castelao
29,Lloret De Mar,ES,89.11,scattered clouds,41.6999,2.8457,Apartaments Guitart
30,Itoman,JP,84.40,few clouds,26.1247,127.6694,Southern Beach Hotel & Resort Okinawa


In [20]:
# 8a. Create the output File (CSV)

output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv

clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [24]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.

info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.

hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.

locations = clean_hotel_df[["Lat", "Lng"]]


In [22]:
# 11a. Add a marker layer for each city to the map. 

#locations = clean_hotel_df[["Lat", "Lng"]]

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))